# Imports

In [46]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [63]:
dataset_name = 'jigsaw unintended bias'
model_name = 'bert'

# change to f'../../results/{dataset_name}' when using inside one of the data subfolders
result_folder = f'../../results/{dataset_name}'
test_csv_filepath = os.path.join(result_folder, 'test.csv')

model_folder = os.path.join(result_folder, model_name) # for this particular model
normal_folder = os.path.join(model_folder, 'normal')
dp_folder = os.path.join(model_folder, 'epsilon 3.0')

result_filepath = os.path.join(normal_folder, 'results.csv')
dp_result_filepath = os.path.join(dp_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [69]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)

df = pd.read_csv(test_csv_filepath)
# df.drop(columns=['comment_text', 'labels'], inplace=True)
# df.to_csv(test_csv_filepath, index=False)
df.head(2)

,id,male,female,white,black
0,7084460,NaN,NaN,NaN,NaN
1,7141509,NaN,NaN,NaN,NaN


In [65]:
result.head(2)

,id,labels,probs,split
0,294256,0,0.234011,train
1,434688,0,0.055698,train


In [66]:
dp_result.head(2)

,id,labels,probs,split
0,7104596,0,0.023603,test
1,7136711,0,0.007524,test


In [70]:
df.fillna(0, inplace=True)

## Split test result

In [71]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

In [55]:
# result = result.drop(columns='id').reset_index().rename({'index':'id'}, axis=1)
# dp_result = dp_result.drop(columns='id').reset_index().rename({'index':'id'}, axis=1)

## Merge

In [73]:
# temporary fix for invalid indexing in the result file
for column in df.columns:
    if column =='id': continue
    result[column]  = df[df['id'].isin(dp_result['id'])][column].values

# result = result.merge(df, on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)

# Evaluation

## Convert probability to prediction

In [87]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [59]:
group_map = {
    'gender': {
        'unprivileged':['female'],
        'privileged':['male']
    },
    'race': {
        'unprivileged':['black'],
        'privileged': ['white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['female', 'male', 'black', 'white']


### Binarize identity and target columns

In [88]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [89]:
bias_results = {
   # make sure your calculate bias method returns bias metrics in the same order
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.973485,0.975852,0.948217,0.797963
1,Equality of Opportunity (w.r.t y = 1),0.976655,0.978125,0.900000,0.903395
2,Equality of Opportunity (w.r.t y = 0),0.991862,0.980068,0.993239,0.938017
3,Equality of Odds,0.984258,0.979097,0.946620,0.920706
4,unprotected-accuracy,0.925058,0.822829,0.930355,0.712221
5,protected-accuracy,0.917614,0.806818,0.913413,0.699491
6,accuracy,0.921336,0.814823,0.921884,0.705856


In [90]:
bias_results.round(3).to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)

## Overall metrics

In [91]:
overall_results = {
    'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'false positive rate']
}

for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    privileged_group = subgroup_map['privileged']
    unprivileged_group = subgroup_map['unprivileged']

    privileged_group_name = ','.join(privileged_group)
    unprivileged_group_name = ','.join(unprivileged_group)

    overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
    overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

    overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
    overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

overall_results['Total'] = calculate_metrics(result, [])
overall_results['Total_DP'] = calculate_metrics(dp_result, [])

overall_results = pd.DataFrame(overall_results) 
overall_results

,metrics,male,male_DP,female,female_DP,white,white_DP,black,black_DP,Total,Total_DP
0,auc,0.942255,0.849390,0.948792,0.850251,0.940519,0.771067,0.960141,0.737525,0.941883,0.894798
1,accuracy,0.917614,0.806818,0.925058,0.822829,0.913413,0.699491,0.930355,0.712221,0.925390,0.909094
2,f1_score,0.560606,0.531034,0.618395,0.515247,0.585366,0.602247,0.607407,0.598165,0.614494,0.553024
3,precision,0.462500,0.420000,0.533784,0.407654,0.493151,0.499069,0.482353,0.545151,0.523332,0.455476
4,recall,0.711538,0.721875,0.734884,0.700000,0.720000,0.759207,0.820000,0.662602,0.744117,0.703742
5,false positive rate,0.065951,0.178013,0.057813,0.158082,0.068646,0.326061,0.061885,0.264078,0.058866,0.073071


In [ ]:
overall_results.round(3).to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)

# Batch benchmarking
Benchmark all models and privacy budgets

In [93]:
for model_name in ['bert']:
    model_folder = os.path.join(result_folder, model_name) # for this particular model
    normal_folder = os.path.join(model_folder, 'normal')
    result_filepath = os.path.join(normal_folder, 'results.csv')

    result = pd.read_csv(result_filepath)
    result = result[result['split']=='test']
    # drop split column
    result.drop(columns=['split'], inplace=True)

    for column in df.columns:
        if column =='id': continue
        result[column]  = df[df['id'].isin(dp_result['id'])][column].values

    result[prediction_column] = result[probability_column] >=0.5
    result = binarize(result, [target_column] + identities)

    
    for epsilon in [3.0, 6.0, 9.0]:
        dp_folder = os.path.join(model_folder, f'epsilon {epsilon}')
        dp_result_filepath = os.path.join(dp_folder, 'results.csv')
        dp_result = pd.read_csv(dp_result_filepath)

        # only calculate test result
        dp_result = dp_result[dp_result['split']=='test']
        dp_result.drop(columns=['split'], inplace=True)
        dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)
        
        dp_result[prediction_column] = dp_result[probability_column] >=0.5
        dp_result = binarize(dp_result, [target_column] + identities)

        bias_results = {
        'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
        'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
        'protected-accuracy', 'accuracy']
        }

        for group_key in group_map.keys():
            subgroup_map = group_map[group_key]
            privileged_group = subgroup_map['privileged']
            unprivileged_group = subgroup_map['unprivileged']

            bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
            bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

        bias_results = pd.DataFrame(bias_results) 
        bias_results.round(3).to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)


        overall_results = {
            'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'false positive rate']
        }

        for group_key in group_map.keys():
            subgroup_map = group_map[group_key]
            privileged_group = subgroup_map['privileged']
            unprivileged_group = subgroup_map['unprivileged']

            privileged_group_name = ','.join(privileged_group)
            unprivileged_group_name = ','.join(unprivileged_group)

            overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
            overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

            overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
            overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

        overall_results['Total'] = calculate_metrics(result, [])
        overall_results['Total_DP'] = calculate_metrics(dp_result, [])

        overall_results = pd.DataFrame(overall_results) 
        overall_results.columns = [col.replace('target_', '') for col in overall_results.columns]
        overall_results.round(3).to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)